# PointLoadCantilever

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

In [1]:
using TopOpt, Makie
import GeometryBasics

┌ Warning: Loaded OpenGL Backend, but OpenGL isn't working
└ @ GLMakie ~/.julia/packages/GLMakie/5aJEt/src/GLMakie.jl:73
┌ Warning: Loaded OpenGL Backend, but OpenGL isn't working
└ @ GLMakie /home/travis/.julia/packages/GLMakie/5aJEt/src/GLMakie.jl:73


### Define the problem

In [2]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0; # downward force

nels = (12, 6, 6) # change to (40, 20, 20) for a more high-res result
problem = PointLoadCantilever(Val{:Linear}, nels, (1.0, 1.0, 1.0), E, v, f);

### Parameter settings

In [3]:
V = 0.3 # volume fraction
xmin = 0.001 # minimum density
rmin = 4.0; # density filter radius

### Define a finite element solver

In [4]:
penalty = TopOpt.PowerPenalty(3.0)
solver = FEASolver(Displacement, Direct, problem, xmin = xmin,
    penalty = penalty);

### Define compliance objective

In [5]:
obj = Objective(TopOpt.Compliance(problem, solver, filterT = DensityFilter,
    rmin = rmin, tracing = true, logarithm = false));

### Define volume constraint

In [6]:
constr = Constraint(TopOpt.Volume(problem, solver, filterT = DensityFilter, rmin = rmin), V);

### Define subproblem optimizer

In [7]:
mma_options = options = MMA.Options(maxiter = 3000,
    tol = MMA.Tolerances(kkttol = 0.001))
convcriteria = MMA.KKTCriteria()
optimizer = MMAOptimizer(obj, constr, MMA.MMA87(),
    ConjugateGradient(), options = mma_options,
    convcriteria = convcriteria);

kkt residual: 0.6999999999999984
ipopt residual: 0.6999999999999984


### Define SIMP optimizer

In [8]:
simp = SIMP(optimizer, penalty.p);

### Solve

In [9]:
x0 = fill(1.0, length(solver.vars))
result = simp(x0);

kkt residual: 0.6999999999999984
ipopt residual: 0.6999999999999984
kkt residual: 3.351836465773589
ipopt residual: 3.351836465773589
kkt residual: 559.6972566557297
ipopt residual: 138.10925876464105
kkt residual: 18.504209525362583
ipopt residual: 18.504209525362583
kkt residual: 3.9032361707277814
ipopt residual: 3.9032361707277814
kkt residual: 3.3087979891761545
ipopt residual: 3.3087979891761545
kkt residual: 3.2556777429958723
ipopt residual: 3.2556777429958723
kkt residual: 2.6396304914560282
ipopt residual: 2.6396304914560282
kkt residual: 2.4517684982577856
ipopt residual: 2.4517684982577856
kkt residual: 1.9706682233656232
ipopt residual: 1.9706682233656232
kkt residual: 1.4411492880930563
ipopt residual: 1.4411492880930563
kkt residual: 1.0884521181240758
ipopt residual: 1.0884521181240758
kkt residual: 0.7356097609906984
ipopt residual: 0.7356097609906984
kkt residual: 0.5721839530094559
ipopt residual: 0.5721839530094559
kkt residual: 0.3715484174080763
ipopt residual: 0.

### Visualize the result using Makie.jl

In [10]:
result_mesh = GeometryBasics.Mesh(problem, result.topology);

mesh(result_mesh);

┌ Info: Makie/AbstractPlotting is caching fonts, this may take a while. Needed only on first run!
└ @ AbstractPlotting /home/travis/.julia/packages/AbstractPlotting/Q2ju2/src/utilities/texture_atlas.jl:115


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*